In [17]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import cartopy.crs as ccrs
from climatetranslation.unit.data import construct_regridders, reduce_height, get_dataset
xr.set_options(display_style="html")

In [18]:
reduce_dict = {0: ['pr'], 2: ['tas', 'tasmin', 'tasmax']}

In [19]:
ds_cam = get_dataset('/datadrive/cam5/nat_hist_transposed_zarr', level_vars=reduce_dict)
ds_had = get_dataset('/datadrive/hadgem3/nat_hist_transposed_zarr', level_vars=reduce_dict)

In [20]:
ds_had

,Array,Chunk
Bytes,86.00 GB,24.88 MB
Shape,"(10, 19440, 192, 288)","(10, 19440, 4, 4)"
Count,6913 Tasks,3456 Chunks
Type,float64,numpy.ndarray
,Array,Chunk
Bytes,86.00 GB,24.88 MB
Shape,"(10, 19440, 192, 288)","(10, 19440, 4, 4)"
Count,6913 Tasks,3456 Chunks
Type,float64,numpy.ndarray
,Array,Chunk


In [95]:
192*288*10*20621*4e-10

4.561035264

In [22]:
ds_cam

,Array,Chunk
Bytes,41.05 GB,1.32 MB
Shape,"(9, 20621, 192, 288)","(1, 20621, 4, 4)"
Count,93313 Tasks,31104 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,41.05 GB,1.32 MB
Shape,"(9, 20621, 192, 288)","(1, 20621, 4, 4)"
Count,93313 Tasks,31104 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [86]:
from scipy.interpolate import interp1d


def translate_quantile_value_single_month(ds, quantile_values, value2quantile=True):

    def interpolate(x, xs, ys):
        if value2quantile:
            fill_value=(0,1)
        else:
            fill_value=(ys[0], ys[-1])
        return interp1d(xs, ys, kind="linear", fill_value=fill_value, bounds_error=False, assume_sorted=True)(x)
    
    return xr.apply_ufunc(
            interpolate,
            ds,
            quantile_values if value2quantile else quantile_values.quantiles,
            quantile_values.quantiles if value2quantile else quantile_values,
            input_core_dims=[['time'], ['quantiles'], ['quantiles']],
            output_core_dims = [['time']],
            exclude_dims = {'quantiles',},
            dask= "allowed",
            vectorize=True
        )


class CDF:
    def __init__(self, quantiles):
        self.quantiles = quantiles
        
    def fit(self, ds):
        self.quantile_values = ds.groupby('time.month').quantile(
            self.quantiles, dim=('time', 'run'), interpolation='linear'
        ).rename(name_dict={'quantile':'quantiles'})
        
    def plot(self, lat, lon, figsize=(6, 18)):
        kn = len(self.quantile_values.keys())
        plt.figure(figsize=figsize)
        for i, k in enumerate(self.quantile_values.keys()):
            plt.subplot(kn, 1, i+1)
            self.quantile_values[k].sel(lat=lat, lon=lon, method='nearest').plot.line(x='quantiles',  add_legend=i==0)
        plt.tight_layout()

    def transform(self, ds):
        qs = self.quantile_values.sel(lat=ds.lat, lon=ds.lon)
        results = []
        for month, group in ds.groupby('time.month'):
            results.append(
                translate_quantile_value_single_month(
                    group, 
                    qs.sel(month=month),
                    value2quantile=True
                )
            )
        return xr.concat(results, dim='time').drop('month').sortby('time')
    
    def inverse_transform(self, ds):
        qs = self.quantile_values.sel(lat=ds.lat, lon=ds.lon)
        results = []
        for month, group in ds.groupby('time.month'):
            results.append(
                translate_quantile_value_single_month(
                    group, 
                    qs.sel(month=month),
                    value2quantile=False
                )
            )
        return xr.concat(results, dim='time').drop('month').sortby('time')
    
    def save(self, filepath):
        self.quantile_values.to_netcdf(filepath)
        
    @classmethod
    def load(cls, filepath):
        if '*' in filepath:
            quantile_values = xr.open_mfdataset(filepath).load()
        else:
            quantile_values = xr.load_dataset(filepath)
        quantiles = quantile_values.quantiles.values
        cdf = CDF(quantiles)
        cdf.quantile_values = quantile_values
        return cdf
            
    
class QauntileMapping:
    def __init__(self, cdfa, cdfb):
        self.cdfa = cdfa
        self.cdfb = cdfb
        
    def transform_a2b(self, ds):
        return self.cdfb.inverse_transform(self.cdfa.transform(ds))
        
    def transform_b2a(self, ds):
        return self.cdfa.inverse_transform(self.cdfb.transform(ds))
        

In [87]:
class Args:
    pass
args = Args()
args.config = "/home/dfulu/repos/climateTranslation/climatetranslation/quantile_mapping/config.yaml"
args.output_zarr = "/home/dfulu/temp_zarr"
args.x2x = "a2b"
args.n_times = 2


config = get_config(args.config)

# unpack a few things for convenience
CDFfilepath = config['output_root']

    
cdf_a = CDF.load(f'{CDFfilepath}/quantiles_a.nc')
cdf_b = CDF.load(f'{CDFfilepath}/quantiles_b.nc')

QM = QauntileMapping(cdf_a, cdf_b)


In [51]:
config

{'number_splits_lat': 20,
 'number_splits_lon': 20,
 'data_zarr_a': '/datadrive/hadgem3/nat_hist_transposed_zarr',
 'data_zarr_b': '/datadrive/cam5/nat_hist_transposed_zarr',
 'level_vars': {0: ['pr'], 2: ['tas', 'tasmin', 'tasmax']},
 'number_of_quantiles': 100,
 'fraction_gap': 0.001,
 'output_root': '/home/dfulu/model_outputs/quantile_mapping_models/v1'}

In [52]:
ds_had = ds_had.isel(lon=slice(0, cdf_a.quantile_values.lon.shape[0]))
ds_cam = ds_cam.isel(lon=slice(0, cdf_b.quantile_values.lon.shape[0]))

In [53]:
cdf_a.quantile_values

<xarray.Dataset>
Dimensions:    (lat: 192, lon: 192, month: 12, quantiles: 100)
Coordinates:
  * month      (month) int64 1 2 3 4 5 6 7 8 9 10 11 12
  * quantiles  (quantiles) float64 0.001 0.01108 0.02116 ... 0.9788 0.9889 0.999
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17 ... 88.12 89.06 90.0
  * lon        (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 235.0 236.2 237.5 238.8
Data variables:
    pr         (month, quantiles, lat, lon) float64 0.0 0.0 ... 6.104e-05
    tas        (month, quantiles, lat, lon) float64 237.1 237.1 ... 263.7 263.7
    tasmax     (month, quantiles, lat, lon) float64 237.8 237.8 ... 266.5 266.5
    tasmin     (month, quantiles, lat, lon) float64 236.3 236.3 ... 260.4 260.4

In [54]:
cdf_b.quantile_values.isel(**common_filt).sel(month=month)

<xarray.Dataset>
Dimensions:    (lat: 4, lon: 4, quantiles: 100)
Coordinates:
    month      int64 1
  * quantiles  (quantiles) float64 0.001 0.01108 0.02116 ... 0.9788 0.9889 0.999
  * lat        (lat) float64 -90.0 -89.06 -88.12 -87.17
  * lon        (lon) float64 0.0 1.25 2.5 3.75
Data variables:
    pr         (quantiles, lat, lon) float64 0.0 0.0 0.0 ... 1.677e-05 1.688e-05
    tas        (quantiles, lat, lon) float64 233.8 234.3 234.4 ... 250.1 249.8
    tasmax     (quantiles, lat, lon) float64 234.2 234.8 234.8 ... 251.3 251.0
    tasmin     (quantiles, lat, lon) float64 233.5 234.0 234.0 ... 248.8 248.4

In [81]:
common_filt = dict(lat=slice(0,4), lon=slice(0,4))
ds_filt = common_filt.copy()
ds_filt.update(dict(run=0, time=slice(None,None)))

for month, group in ds_cam.groupby('time.month'):
    results = translate_quantile_value_single_month(
        group.isel(**ds_filt), 
        cdf_b.quantile_values.isel(**common_filt).sel(month=month), 
        value2quantile=True
    )
    break


/anaconda/envs/py37_fresh/lib/python3.7/site-packages/xarray/core/indexing.py:1343: PerformanceWarning: Slicing is producing a large chunk. To accept the large
chunk and silence this warning, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': False}):
    ...     array[indexer]

To avoid creating the large chunks, set the option
    >>> with dask.config.set(**{'array.slicing.split_large_chunks': True}):
    ...     array[indexer]
  return self.array[key]


In [82]:
results

<xarray.Dataset>
Dimensions:  (lat: 4, lon: 4, time: 1767)
Coordinates:
    run      int64 1
  * lon      (lon) float32 0.0 1.25 2.5 3.75
  * time     (time) object 1959-01-01 12:00:00 ... 2015-01-31 12:00:00
  * lat      (lat) float32 -90.0 -89.057594 -88.11518 -87.172775
    month    int64 1
Data variables:
    pr       (lat, lon, time) float64 0.3626 0.08619 0.04508 ... 0.2732 0.8858
    tas      (lat, lon, time) float64 0.7571 0.6047 0.4998 ... 0.198 0.1199
    tasmax   (lat, lon, time) float64 0.7024 0.6481 0.4545 ... 0.1769 0.1114
    tasmin   (lat, lon, time) float64 0.7827 0.5725 0.5487 ... 0.2168 0.1566

In [88]:
ds_mapped = QM.transform_a2b(ds_had.isel(**common_filt))

In [92]:
ds_mapped

<xarray.Dataset>
Dimensions:  (lat: 4, lon: 4, run: 10, time: 19440)
Coordinates:
  * run      (run) int64 1 2 3 4 5 6 7 8 9 10
  * lon      (lon) float32 0.0 1.25 2.5 3.75
  * lat      (lat) float32 -90.0 -89.057594 -88.11518 -87.172775
  * time     (time) object 1960-01-01 12:00:00 ... 2013-12-30 12:00:00
Data variables:
    pr       (run, lat, lon, time) float64 8.099e-07 4.211e-06 ... 4.737e-07
    tas      (run, lat, lon, time) float64 245.3 246.7 245.7 ... 242.0 242.9
    tasmax   (run, lat, lon, time) float64 246.0 246.8 246.4 ... 242.7 243.9
    tasmin   (run, lat, lon, time) float64 244.5 246.4 245.4 ... 240.7 242.3

In [96]:
ds_cam

,Array,Chunk
Bytes,27.37 GB,1.32 MB
Shape,"(9, 20621, 192, 192)","(1, 20621, 4, 4)"
Count,114049 Tasks,20736 Chunks
Type,float32,numpy.ndarray
,Array,Chunk
Bytes,27.37 GB,1.32 MB
Shape,"(9, 20621, 192, 192)","(1, 20621, 4, 4)"
Count,114049 Tasks,20736 Chunks
Type,float32,numpy.ndarray
,Array,Chunk


In [99]:
array = ds_cam.isel(lat=slice(0,12), lon=slice(0, 12))
array.map_blocks(
    QM.transform_a2b,
    template=array,
) 

<xarray.Dataset>
Dimensions:  (lat: 12, lon: 12, run: 9, time: 20621)
Coordinates:
  * run      (run) int64 1 2 3 4 5 7 8 9 10
  * lon      (lon) float64 0.0 1.25 2.5 3.75 5.0 ... 8.75 10.0 11.25 12.5 13.75
  * time     (time) object 1959-01-01 12:00:00 ... 2015-06-30 12:00:00
  * lat      (lat) float64 -90.0 -89.06 -88.12 -87.17 ... -81.52 -80.58 -79.63
Data variables:
    pr       (run, time, lat, lon) float32 dask.array<chunksize=(1, 20621, 4, 4), meta=np.ndarray>
    tas      (run, time, lat, lon) float32 dask.array<chunksize=(1, 20621, 4, 4), meta=np.ndarray>
    tasmax   (run, time, lat, lon) float32 dask.array<chunksize=(1, 20621, 4, 4), meta=np.ndarray>
    tasmin   (run, time, lat, lon) float32 dask.array<chunksize=(1, 20621, 4, 4), meta=np.ndarray>

In [14]:

mode = 'w-'
append_dim = None

N_times = 10

with progressbar.ProgressBar(max_value=N_times) as bar:
        
    for i in range(0, N_times, args.n_times):

        # transform through network 
        ds_mapped = QM.transform_a2b(ds.isel(time=slice(i, min(i+args.n_times, N_times))).compute())

        # fix chunking
        ds_mapped = ds_mapped.chunk(dict(run=1, time=1, lat=-1, lon=-1))

        # append to zarr
        ds_mapped.to_zarr(
            output_zarr, 
            mode=mode, 
            append_dim=append_dim,
            consolidated=True
        )

        # update progress bar and change modes so dat can be appended
        bar.update(i)
        mode, append_dim='a', 'time'
            
    bar.update(N_times)

100% (10 of 10) |########################| Elapsed Time: 0:00:00 ETA:  00:00:00


KeyboardInterrupt: 